In [3]:
!pip install torch

  Using cached torch-2.4.0-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/197.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/197.9 MB 7.6 MB/s eta 0:00:27
   ---------------------------------------- 0.5/197.9 MB 5.6 MB/s eta 0:00:35
   ---------------------------------------- 0.6/197.9 MB 4.8 MB/s eta 0:00:41
   ---------------------------------------- 0.8/197.9 MB 4.9 MB/s eta 0:00:41
   ---------------------------------------- 1.0/197.9 MB 4.6 MB/s eta 0:00:43
   ---------------------------------------- 1.2/197.9 MB 4.6 MB/s eta 0:00:43
   ---------------------------------------- 1.4/197.9 MB 4.5 MB/s eta 0:00:44
   ---------------------------------------- 1.6/197.9 MB 4.4 MB/s eta 0:00:45
   --------------------------------

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    unknown package:
        Expected sha256 97730014da4c57ffacb3c09298c6ce05400606e890bd7a05008d13dd086e46b1
             Got        540f21f72674a03156c5c62d99d166192e6ec3993fb4bef76e77d0399c4e98f6


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import cv2
import numpy as np

import os

In [5]:
import urllib.request

# URLs to download YOLOv3 files
yolo_config_url = "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
yolo_weights_url = "https://pjreddie.com/media/files/yolov3.weights"
yolo_classes_url = "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

# Local paths to save the files
config_path = "yolov3.cfg"
weights_path = "yolov3.weights"
classes_path = "coco.names"

# Download the YOLOv3 configuration file
urllib.request.urlretrieve(yolo_config_url, config_path)

# Download the YOLOv3 weights file
urllib.request.urlretrieve(yolo_weights_url, weights_path)

# Download the YOLOv3 classes file
urllib.request.urlretrieve(yolo_classes_url, classes_path)

print("Downloaded YOLOv3 files.")


Downloaded YOLOv3 files.


In [19]:
classes_path = "train/_classes.txt"

In [20]:
classes_path

'train/_classes.txt'

In [21]:
# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
classes = []
with open(classes_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [22]:
len(classes)

27

In [23]:
def detect_objects(img_path):
    # Load image
    print(f"Reading image from {img_path}")
    img = cv2.imread(img_path)
    if img is None:
        print("Error: Image not found or unable to load.")
        return

    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Information to show on screen
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Debugging statements to check lengths
    print(f"Boxes: {len(boxes)}, Confidences: {len(confidences)}, Class IDs: {len(class_ids)}, Indexes: {len(indexes)}")
    print(f"Classes: {len(classes)}, Output Layers: {len(output_layers)}")

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            class_id = class_ids[i]
            if class_id >= len(classes):
                print(f"Warning: class_id {class_id} is out of range for classes list.")
                continue
            label = str(classes[class_id])
            color = (0, 255, 0)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 10), font, 1, color, 2)

    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [24]:
# Detect objects in an example image
detect_objects("train/first_img.jpg.jpg")

Reading image from train/first_img.jpg.jpg
Boxes: 2, Confidences: 2, Class IDs: 2, Indexes: 1
Classes: 27, Output Layers: 3


In [2]:
len(classes)

NameError: name 'classes' is not defined